# Homework 4 - Group #24
## 1) Does basic house information reflect house's description?

Our goal is to implement two clustering and compare the results. We create two datasets and each of them will be filled by data that we scraped.

First of all, we import the following libraries.

In [1]:
import time # For time sleep() Method
import pandas as pd
import numpy as np
import requests #requests.get
from urllib.request import urlopen
from bs4 import BeautifulSoup

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
import io
import json

### Scraping & Datasets
Web scraping using **Beautiful Soup** library.

#### 1) Information

In [3]:
def price(d):
    try:
        price = d[0].contents[0].replace("\n", "").replace("€ ","").strip()
    except:
        price = d[0].contents[0].get_text().replace("\n", "").replace("€ ","").strip()

    return price

In [4]:
def locali(d):
    local = d[1].get_text().replace("\xa0","").replace("+", "").replace("locali", "").strip()
    return local

In [5]:
def superficie(d):
    superf = d[2].get_text().replace("da ","").replace("\xa0m2","").replace("superficie","").strip()
    return superf

In [6]:
def bagni(d):
    bath = d[3].get_text().replace("\xa0","").replace("+", "").replace("bagni", "").strip()
    return bath

In [7]:
def piano(d):
    floor = d[4].get_text().replace("\xa0", "").replace("\n","").replace("T","0").replace("piano","").strip()
    return floor

In [10]:
print ("Start : %s" % time.ctime())
numb_ann = 10
df1 = pd.DataFrame(columns=['Price', 'Locali', 'Superficie', 'Bagni', 'Piano'] )
count = 1
i = 1

while count < numb_ann:
    url = "https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag="+str(i)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    mydivs = soup.find_all("div", {"class": "listing-item_body--content"})
    i +=1
    for a in mydivs:
        ann = "announcement_"+ str(count)
        data = a.find_all("li", {"class":"lif__item"})
        
        if len(data) == 6 and count <= numb_ann:
            pr = price(data)
            loc = locali(data)
            mq = superficie(data)
            bath = bagni(data) 
            floor = piano(data)
            l = [pr, loc, mq, bath, floor]

            df = pd.DataFrame([l], columns=['Price', 'Locali', 'Superficie', 'Bagni', 'Piano'])
            count +=1
            df1 = pd.concat([df1, df], ignore_index=True) 
            
    # time.sleep(720)
print ("End : %s" % time.ctime())

Start : Mon Dec  3 18:42:06 2018
End : Mon Dec  3 18:42:08 2018


In [23]:
df1.head()

,Price,Locali,Superficie,Bagni,Piano
0,225.000,2,50,1,1
1,395.000,5,130,2,1
2,3.700.000,5,250,3,5
3,850.000,5,175,2,A
4,279.000,3,110,1,0


In [28]:
len(df1)

10

#### 2) Description

In [12]:
all_links = soup.find_all('a')
links = []

for link in all_links:
    if link.get('href')!= None:
        links.append(link.get("href"))

need_links = []
for i in links:
    if i[-4:] == 'html':
        need_links.append(i)   

In [16]:
stop_words = set(stopwords.words('italian')) # we take the stopwords we have to delete
tokenizer = RegexpTokenizer(r'\w+')
ps = PorterStemmer()
description_words = []

for link in need_links[:-1]:
    if link[0:5] == 'https':
        site = urlopen(link)
        page = BeautifulSoup(site, 'lxml')
        
        description = page.find_all('div', class_="col-xs-12 description-text text-compressed")
        description = ''.join(str(description))
        
        words = word_tokenize(description)
        words_without_stop_words = ["" if word in stop_words else word for word in words] #we delete the stopwords
        new_words = " ".join(words_without_stop_words)
        b = tokenizer.tokenize(new_words)

        l_stem_words = []
        # with a for loop we stem all the words in the input, and store them in a list
        for word in b:
            l_stem_words.append(ps.stem(word))
        # here we transform the list in a string
        stem_words = ' '.join(l_stem_words).split()
        description_words.append(stem_words)
print(description_words)

[['div', 'aria', 'expand', 'fals', 'class', 'col', 'xs', '12', 'descript', 'text', 'text', 'compress', 'role', 'contentinfo', 'div', 'papillo', 'eur', 'elegant', 'complesso', 'residenzial', 'rifinitissimo', 'bilocal', 'composto', 'soggiorno', 'angolo', 'cottura', 'stanza', 'letto', 'bagno', 'ampio', 'balcon', 'rifinitur', 'pregio', 'pavimenti', 'parquet', 'gree', 'infissi', 'legno', 'vetro', 'camera', 'port', 'noce', 'grate', 'salon', 'riscaldamento', 'termoautonomo', 'caldaia', 'centralizzata', 'aria', 'condizionata', 'videocitofono', 'porta', 'blindata', 'serramenti', 'elettrici', 'chiusura', 'centralizzata', 'antenna', 'satellitar', 'isolamento', 'termo', 'acustico', 'pannelli', 'solari', 'fotovoltaici', 'rilevatori', 'elettronici', 'ga', 'tutt', 'camer', 'fornit', 'impianto', 'antifurto', 'presa', 'antenna', 'satellitar', 'presa', 'telefonica', 'br', 'br', 'orari', 'lunedi', 'chiusi', 'br', 'martedi', '10', '00', '17', '00', 'br', 'mercoledi', '10', '00', '17', '00', 'br', 'hr', 'g

In [18]:
def vocabulary_creation(word_list, previous_vocabulary):
    #create an new dictionary from a previous one which could be empty
    vocabulary=previous_vocabulary
    #obtain last index of previous dictionary and set first index for 1st different element in this vocabulary
    if len(previous_vocabulary)==0:
        number=1
    else:
        number=previous_vocabulary[list(previous_vocabulary)[len(previous_vocabulary)-1]]+1
    #adding new words to the vocabulary and omitting repeated ones
    for lista in word_list:
        for word in lista:
            index=list(vocabulary.keys())
            if word in index:
                pass
            else:
                vocabulary[word]=number
                number+=1
    return vocabulary

In [20]:
vocabulary={}
aux=0
# loop to generate a vocabulary from all the documents with title, description and city
vocabulary_text = vocabulary_creation(description_words, vocabulary)
vocabulary = vocabulary_text

In [26]:
print(vocabulary)

{'div': 1, 'aria': 2, 'expand': 3, 'fals': 4, 'class': 5, 'col': 6, 'xs': 7, '12': 8, 'descript': 9, 'text': 10, 'compress': 11, 'role': 12, 'contentinfo': 13, 'papillo': 14, 'eur': 15, 'elegant': 16, 'complesso': 17, 'residenzial': 18, 'rifinitissimo': 19, 'bilocal': 20, 'composto': 21, 'soggiorno': 22, 'angolo': 23, 'cottura': 24, 'stanza': 25, 'letto': 26, 'bagno': 27, 'ampio': 28, 'balcon': 29, 'rifinitur': 30, 'pregio': 31, 'pavimenti': 32, 'parquet': 33, 'gree': 34, 'infissi': 35, 'legno': 36, 'vetro': 37, 'camera': 38, 'port': 39, 'noce': 40, 'grate': 41, 'salon': 42, 'riscaldamento': 43, 'termoautonomo': 44, 'caldaia': 45, 'centralizzata': 46, 'condizionata': 47, 'videocitofono': 48, 'porta': 49, 'blindata': 50, 'serramenti': 51, 'elettrici': 52, 'chiusura': 53, 'antenna': 54, 'satellitar': 55, 'isolamento': 56, 'termo': 57, 'acustico': 58, 'pannelli': 59, 'solari': 60, 'fotovoltaici': 61, 'rilevatori': 62, 'elettronici': 63, 'ga': 64, 'tutt': 65, 'camer': 66, 'fornit': 67, 'im

In [27]:
len(vocabulary)

1189

In [24]:
with open('data/vocabulary.tsv', 'w') as f: #vocabulary with all words is written in file 'vocabulary.tsv'
    json.dump(vocabulary, f) 
f.close()

In [30]:
#functions defined in order to calculate the tfIdf coefficient
def N(df1):
    #number of documents in the collection
    return len(df1)

def df_t(word, df1):
    #number of documents in the collection that contain a term t
    counter=0
    for lista in df1:
        if word in lista:
            counter+=1
    return counter

def tf_t_d(word,lista):
    #term frequency:number of ocurrence of term t in document d
    return lista.count(word)

def id_f_t(word, df1):
    #inverse document frequency of a term t
    return log(len(df1)/df_t(word, df1),10)

def tf_idf(word,lista,df1):
    #tf_idf of a term in a document of a collection N
    return tf_t_d(word,lista)*id_f_t(word, df1)

def inverted_index_creation_tfIdf(vocabulary, df1):  
    #function that creates inverted_index with coefficient tfIdf from all the documents
    inverted_index={}
    n=1
    
    for lista in df1:          
         #loop for each word of the document create a new key if the word is not in the dictionary 
         #add the number of the document to an existing key is the word is in the dictionary
         #tfIdf coefficient of each term is also added
        for word in set(lista):
    
            index=list(vocabulary.values())
            if vocabulary[word] in list(inverted_index.keys()):
                inverted_index[vocabulary[word]]=(inverted_index[vocabulary[word]])+[("announcement_"+str(n),tf_idf(word,lista,df1))]
            else:
                inverted_index[int(vocabulary[word])]=[("announcement_"+str(n),tf_idf(word,lista,df1))]
        
        n+=1

    with open('data/inverted_index_tfIdf.tsv', 'w') as f1: #write the inverted index in a file called "inverted_index_tfIdf.tsv"
        json.dump(inverted_index, f1)
        f1.close()
    return inverted_index 



We need to calculate the **TF-IDF** of all the words, that is defined as the "term frequency" times the "inverse document frequency" where:
- "term frequency" is the ratio between the number of the term occurencies in the document and the total number of words in the document;
- "inverse document frequency" is the logarithm of the ratio between the total number of documents and the number of documents containing the term (plus 1 to avoid division by zero).

IDF is indipendent from the specific document, thus we can calculate it once and use it when we need it. Each term will have a single IDF.

In [32]:
# inverted index 
inverted_index = {}  

for key, value in .items():
    for i in sorted(list(set(value))):
        if (vocabulary[i] in inverted_index):
            inverted_index[vocabulary[i]] = inverted_index[vocabulary[i]] + [key]
        else:
            inverted_index[vocabulary[i]] = [key]

NameError: name 'docd' is not defined

In [ ]:
# dictionary with idf of all word
idf = {}
n_ann = len(df1)

for term_id, doc in inverted_docd.items():
    idf[term_id] = pd.np.log(n_ann/(1+len(doc)))